In [ ]:
import numpy as np
import logging
import utils
import problem_gen
from titanq import Model, Vtype, Target, S3Storage

## Setting Credentials
The user should configure their TitanQ API key here. For very large problems, the user must also configure an AWS Access key, AWS Secret Access key and AWS Bucket Name.

In [ ]:
logging.getLogger('botocore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

TITANQ_DEV_API_KEY = "Your API key"

# Specify AWS keys and bucket name for solving very large problems
# AWS_ACCESS_KEY = "Your Access key"
# AWS_SECRET_ACCESS_KEY = "Your secret access key"
# AWS_BUCKET_NAME = "Your bucket name"

## Problem Definition

Read instance file as a dictionary where the keys are the job id, and the values are a list of tuples (machine_id, processing_time).

In [ ]:
# Reading the JSSP instance file
problem = dict(utils.read_instance("instances/jssp_1"))

# Some useful variables
num_tasks = problem_gen.get_num_tasks(problem)
max_time = problem_gen.get_max_time(problem)
num_machines = problem_gen.get_num_machines(problem)
num_jobs = len(problem.keys())

num_rows = num_tasks
num_cols = max_time

## Model Definition

Defines hyperparameters used in the QUBO formulation as well as the QUBO matrix:

- The first three hyperparameters (a,b,c) correspond to the constraints of the problem.

- The fourth and fifth hyperparameters (d,e) correspond to the objective function of the problem.

In [ ]:
# Hyperparameters for the constraints
a = 4000
b = 4000
c = 4000

# Hyperparameters for the objective function
d = 20
e = 20

# Calling the function that generates the QUBO matrix
qubo = problem_gen.generate_qubo(problem, a, b, c, d, e)
size = len(qubo)

## Construction of the Input for the TitanQ SDK

Here we construct both the bias (the linear terms) and the weights (the quadratic terms) from the QUBO matrix.

In [ ]:
# Construction of the bias vector
bias = np.array(qubo, dtype=np.float32).diagonal()

# Construction of the weight matrix
weights = np.array(qubo,dtype=np.float32)

# The diagonal of the weight matrix should contain only null values
np.fill_diagonal(weights,0)

## Additional Parameter Setup for the TitanQ SDK

Define the variable name along with the variable type.

- Vtype.BINARY if the variable is binary.

- Vtype.BIPOLAR if the variable is bipolar (Coming soon).

Set the weights and bias vector defined above previously into the SDK.

Specify whether to minimize or maximize the Hamiltonian as the objective of the solver (Set to Target.MINIMIZE by default).

Additional hyperparameters which could be tuned include:

- *beta* = Scales the problem by this factor (inverse of temperature). A lower *beta* allows for easier escape from local minima, while a higher *beta* is more likely to respect penalties and constraints.

- *coupling_mult* = Strength of the minor embedding for the titanQ specific hardware.

- *timeout_in_secs* = Maximum runtime of the solver in seconds.

- *num_chains* = Number of parallel runs executed by the solver. A larger number of parallel runs generally leads to higher quality solutions.

In [ ]:
#############
# TitanQ SDK
#############
model = Model(
    api_key=TITANQ_DEV_API_KEY,
    # Insert storage_client parameter and specify corresponding AWS keys and bucket name for solving very large problems
    # storage_client=S3Storage(
    #     access_key=AWS_ACCESS_KEY,
    #     secret_key=AWS_SECRET_ACCESS_KEY,
    #     bucket_name=AWS_BUCKET_NAME,
    # )
)
model.add_variable_vector('x', size, Vtype.BINARY)
model.set_objective_matrices(weights, bias, Target.MINIMIZE)
response = model.optimize(timeout_in_secs=10, coupling_mult=0.75)

## Printing Results

The solution and corresponding Ising energy can be accessed as shown below.

In [ ]:
print("--- RESULTS ---")
print("     x:", np.array(response.x,dtype=int).tolist())
print("energy:", response.ising_energy)

## Solution Visualization

The solution is plotted below as a GANTT chart where each color defines a certain job and each row define a machine id.

In [ ]:
solution = utils.from_sigma_to_solution(np.reshape(response.x,(num_rows,num_cols)),problem)

utils.draw_solution(problem,solution,x_max=max_time)